In [1]:
#CONDA ENV base_conda (python 3.9.7)
#Import packages
#---------------------------------------
import sys
import os
import glob
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm


#Import your modules
#---------------------------------------
import te_rna_f as ter
sys.path.insert(1, '/cndd3/dburrows/CODE/admin_tools/')
from admin_tools import admin_functions as adm

# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
s_fig = '/cndd3/dburrows/FIGS/'

%load_ext autoreload
sys.version


'3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) \n[GCC 9.4.0]'

In [9]:
TEs = pd.read_csv('/cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.merge.rerr.bed',sep='\t',
                  names=['chr','start','end','TEtype','TEfamily','strand','TEclass','length','TSS_start','TSS_end'])

In [10]:
#Filter for most highly expressed families
df = pd.read_csv('/cndd/dburrows/DATA/te/rna/PE.counts/DE/ATEM_CPM.csv', index_col=0)
ind = np.mean(df[df.index.str.contains('L1')], axis=1) > 1 #indeces of ones to keep
l1_keep = df[df.index.str.contains('L1')][ind].index.values

#higher threshold for Alu
ind = np.mean(df[df.index.str.contains('Alu')], axis=1) > 300
sine_keep = list(df[df.index.str.contains('Alu')][ind].index.values)
sine_keep = list(np.append(sine_keep,  df[df.index.str.contains('SVA')] [np.mean(df[df.index.str.contains('SVA')], axis=1) > 1 ].index.values ))
l1_keep, sine_keep

(array(['L1PA6', 'L1PA4', 'L1PA7', 'L1PA2', 'L1PA5', 'L1PA8A', 'L1PA8',
        'L1PA10', 'L1PA3', 'L1HS'], dtype=object),
 ['AluJo',
  'AluJr',
  'AluSx',
  'AluSz',
  'AluY',
  'AluJb',
  'AluSx1',
  'SVA_D',
  'SVA_F',
  'SVA_B',
  'SVA_E',
  'SVA_C',
  'SVA_A'])

In [ ]:
%autoreload
L1 = TEs.loc[TEs['TEfamily'].isin(l1_keep) & (TEs['chr']!='chrY') & (~TEs['chr'].str.contains('_'))].reset_index()
SINE = TEs.loc[TEs['TEfamily'].isin(sine_keep) & (TEs['chr']!='chrY') & (~TEs['chr'].str.contains('_'))].reset_index()

SINE_bin = ter.bin_bed(SINE, binsize=100, upstream=8000, downstream=8000)
SINE_bin = SINE_bin.sort_values(['chr', 'start'])
# SINE_bin.to_csv('/cndd/dburrows/DATA/te/rna/PE.genomic.bins/SINE_bins.bed',sep='\t',header=False,index=False)

In [ ]:
#bedtools sort -g /cndd/emukamel/DrachevaLiu_PsychENCODE_SCZ/L1_bins/hg38.chroms -i /cndd/dburrows/DATA/te/rna/PE.genomic.bins/SINE_bins.bed > /cndd/dburrows/DATA/te/rna/PE.genomic.bins/SINE_bins.sorted.bed


In [11]:
SINE

,index,chr,start,end,TEtype,TEfamily,strand,TEclass,length,TSS_start,TSS_end
0,3399,chr1,31436,31733,Alu,AluJo,+,SINE,297,31386,31486
1,3400,chr1,39624,39924,Alu,AluSx,+,SINE,300,39574,39674
2,3402,chr1,76893,77201,Alu,AluSz,+,SINE,308,76843,76943
3,3404,chr1,90921,91213,Alu,AluY,+,SINE,293,90871,90971
4,3405,chr1,101056,101352,Alu,AluSz,+,SINE,298,101006,101106
...,...,...,...,...,...,...,...,...,...,...,...
560432,942895,chrX,138282068,138283466,SVA,SVA_B,-,Retroposon,1342,138283416,138283516
560433,942896,chrX,147991652,147993416,SVA,SVA_C,-,Retroposon,1355,147993366,147993466
560434,942897,chrX,151598822,151600317,SVA,SVA_B,-,Retroposon,1382,151600267,151600367
560435,942898,chrX,155223200,155224623,SVA,SVA_B,-,Retroposon,1359,155224573,155224673


In [8]:
SINE_bin

,chr,start,end,TE_start,TE_end,strand,TE_type,TE_family,TE_id
0,chr1,23436,23536,31436,31733,+,Alu,AluJo,chr1_AluJo_23436_bin-8000
1,chr1,23536,23636,31436,31733,+,Alu,AluJo,chr1_AluJo_23536_bin-7900
2,chr1,23636,23736,31436,31733,+,Alu,AluJo,chr1_AluJo_23636_bin-7800
3,chr1,23736,23836,31436,31733,+,Alu,AluJo,chr1_AluJo_23736_bin-7700
4,chr1,23836,23936,31436,31733,+,Alu,AluJo,chr1_AluJo_23836_bin-7600
...,...,...,...,...,...,...,...,...,...
89524644,chrX,156025271,156025371,156017344,156017671,-,Alu,AluJb,chrX_AluJb_156025271_bin-7600
89524643,chrX,156025371,156025471,156017344,156017671,-,Alu,AluJb,chrX_AluJb_156025371_bin-7700
89524642,chrX,156025471,156025571,156017344,156017671,-,Alu,AluJb,chrX_AluJb_156025471_bin-7800
89524641,chrX,156025571,156025671,156017344,156017671,-,Alu,AluJb,chrX_AluJb_156025571_bin-7900


# Make new directories

In [ ]:
#convert below text into list of strings

CZI10_NOVA_220113  CZI13_NOVA_211107  CZI18_NOVA_220113  CZI21_NOVA_220113  CZI2_NOVA_210611  CZI5_NOVA_210816
CZI11_NOVA_220113  CZI15_NOVA_220113  CZI1_NOVA_210502   CZI22_NOVA_220113  CZI3_NOVA_210611  CZI8_NOVA_210905
CZI12_NOVA_211107  CZI17_NOVA_220113  CZI20_NOVA_220113  CZI23_NOVA_220113  CZI4_NOVA_210816  CZI9_NOVA_210905

In [7]:
#avoid CZI14, CZI16, CZI19 
in_path= "/cndd3/dburrows/DATA/te/rna/PE.bam/"
out_path = "/cndd/dburrows/DATA/te/rna/PE.counts/TET/"

samp_l = glob.glob(in_path+'/*Samp*')

for s in samp_l:
    #find base directory of s
    base = os.path.basename(s)
    if not os.path.exists(out_path+'/'+base):
        os.mkdir(out_path+'/'+base)
#5, 8, 9

In [93]:
#symlink files 
curr = glob.glob("/cndd/dburrows/DATA/te/rna/PE.bam_multi20/" + '*Samp*')
nex = glob.glob('/datasets/Public_Datasets/Dracheva_PsychEncode_development/processed/rna_seq/' + '*-*')

for c in curr:
    short = os.path.basename(c).split('_')[-1]
    prac = [i for i in nex if short in i]
    if len(prac) == 1:
         os.symlink(prac[0] + '/Aligned.sortedByCoord.out.bam', c + '/Aligned.sortedByCoord.out.bam')
    elif len(prac) < 1:
        short = short.upper()
        prac = [i for i in nex if short in i]
        if len(prac) == 1:
            os.symlink(prac[0] + '/Aligned.sortedByCoord.out.bam', c + '/Aligned.sortedByCoord.out.bam')
        else:
            if short == 'HCT16HECA028-GABA':
                short = 'HCK16HECA028-GABA'
            if short == 'HCT16HECA028-GLU':
                short = 'HCK16HECA028-GLU'
            prac = [i for i in nex if short in i]
            os.symlink(prac[0] + '/Aligned.sortedByCoord.out.bam', c + '/Aligned.sortedByCoord.out.bam')




In [105]:
#Delete all files in directory
os.chdir('/cndd3/dburrows/DATA/te/rna/CZI.counts/ATEM/pseudobulk')
curr = glob.glob('**')
for c in curr:
    #remove all contents of each directory
    shutil.rmtree(c)


# Check for log files

In [37]:
import glob
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/')
curr_ = glob.glob('*Sa*')
for c in curr_:
    os.chdir(c)
    atem = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + c + '/*ATEM*')
    log = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + c + '/*log*')
    if not atem or not log:
        print(c)
    if log:
        if 'All reads accounted for' not in np.load(log[0]):
            print(c)

    os.chdir('..')

In [28]:
import numpy as np
'All reads accounted for' in np.load(log[0])

True

# Save config files for snakemake

In [1]:
my_dict = {
    "in_path": "/cndd3/dburrows/DATA/te/rna/PE.bam", 
    "out_path": "/cndd/dburrows/DATA/te/rna/PE.counts/TET"
}

In [5]:
import os
import glob
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/TET')
curr_dict = my_dict.copy()
curr_dict['ID'] = glob.glob('*Samp*')
adm.make_config(curr_dict, '/cndd3/dburrows/CODE/te_rna_ageing/snakemake/workflow_benchmark.TET/config.json')

Write successful


In [98]:
curr_dict

{'in_path': '/cndd/dburrows/DATA/te/rna/PE.bam_multi20',
 'out_path': '/cndd/dburrows/DATA/te/rna/PE.counts/RNA-mut',
 'ID': ['Sample_5570-GABA',
  'Sample_5086-GLU',
  'Sample_1823-GABA',
  'Sample_6285-GABA',
  'Sample_1277-GABA',
  'Sample_5976-GABA',
  'Sample_5161-GABA',
  'Sample_4411-GABA',
  'Sample_4369-GLU',
  'Sample_5326-GABA',
  'Sample_HctYPA008-GABA',
  'Sample_5309-GLU',
  'Sample_4411-GLU',
  'Sample_5871-GLU',
  'Sample_5077-GABA',
  'Sample_198034-1435-GABA',
  'Sample_5086-GABA',
  'Sample_Hct15HBNA032-GABA',
  'Sample_5936-GLU',
  'Sample_4337-GABA',
  'Sample_1241-GLU',
  'Sample_5184-GABA',
  'Sample_5918-GABA',
  'Sample_5643-GABA',
  'Sample_5387-GLU',
  'Sample_5976-GLU',
  'Sample_4725-GLU',
  'Sample_4428-GABA',
  'Sample_1539-GLU',
  'Sample_4332-GABA',
  'Sample_4379-GABA',
  'Sample_1848-GABA',
  'Sample_179134-1401-GABA',
  'Sample_Hct16HCQA020-GLU',
  'Sample_5643-GLU',
  'Sample_4321-GLU',
  'Sample_5744-GABA',
  'Sample_5401-GABA',
  'Sample_5293-GABA

In [38]:
[(curr[i], p) for i in range(len(curr)) if p in curr[i]]

[('5451-GABA_ATGGCATG-GGTACCTT_HKVTWDSX3', '5451-GABA')]

In [42]:
prac = [i.split('_')[-1] for i in curr_dict['ID']]
os.chdir('/datasets/Public_Datasets/Dracheva_PsychEncode_development/processed/rna_seq/')
curr = glob.glob('*-*')
for p in prac:
    print(p)
    print([(curr[i], p) for i in range(len(curr)) if p in curr[i]])

5570-GABA
[('5570-GABA_TCTCTACT-GAACCGCG_HKVTWDSX3', '5570-GABA')]
5086-GLU
[('5086-GLU_TCTGTTGG-CCATTCGA_HKVTWDSX3', '5086-GLU')]
1823-GABA
[('1823-GABA_GATCTATC-AGCCTCAT_HKVTWDSX3', '1823-GABA')]
6285-GABA
[('6285-GABA_ACTAAGAT-CCGCGGTT_HKVTWDSX3', '6285-GABA')]
1277-GABA
[('1277-GABA_AAGATACT-ATGTAAGT_HKVTWDSX3', '1277-GABA')]
5976-GABA
[('5976-GABA_GCCACAGG-ATGGCATG_HKVTWDSX3', '5976-GABA')]
5161-GABA
[('5161-GABA_AGGTTATA-CGGAACTG_HKVTWDSX3', '5161-GABA')]
4411-GABA
[('4411-GABA_CAATTAAC-ATATCTCG_HKVTWDSX3', '4411-GABA')]
4369-GLU
[('4369-GLU_TAAGGTCA-CTACGACA_HKVTWDSX3', '4369-GLU')]
5326-GABA
[('5326-GABA_ATGTAAGT-CATAGAGT_HKVTWDSX3', '5326-GABA')]
HctYPA008-GABA
[]
5309-GLU
[('5309-GLU_GCGCAAGC-CGGCGTGA_HKVTWDSX3', '5309-GLU')]
4411-GLU
[('4411-GLU_TGGCCGGT-GCGCTCTA_HKVTWDSX3', '4411-GLU')]
5871-GLU
[('5871-GLU_ACGCACCT-CCTTCACC_HKVTWDSX3', '5871-GLU')]
5077-GABA
[('5077-GABA_ACACTAAG-ATATGGAT_HKVTWDSX3', '5077-GABA')]
198034-1435-GABA
[('198034-1435-GABA_TATCGCAC-ACACTAAG_HKVT

In [18]:
curr_dict

{'in_path': '/cndd3/dburrows/DATA/te/rna/PE.bam',
 'out_path': '/cndd/dburrows/DATA/te/rna/PE.counts/RNA-mut',
 'ID': ['Sample_5570-GABA',
  'Sample_5086-GLU',
  'Sample_1823-GABA',
  'Sample_6285-GABA',
  'Sample_1277-GABA',
  'Sample_5976-GABA',
  'Sample_5161-GABA',
  'Sample_4411-GABA',
  'Sample_4369-GLU',
  'Sample_5326-GABA',
  'Sample_HctYPA008-GABA',
  'Sample_5309-GLU',
  'Sample_4411-GLU',
  'Sample_5871-GLU',
  'Sample_5077-GABA',
  'Sample_198034-1435-GABA',
  'Sample_5086-GABA',
  'Sample_Hct15HBNA032-GABA',
  'Sample_5936-GLU',
  'Sample_4337-GABA',
  'Sample_1241-GLU',
  'Sample_5184-GABA',
  'Sample_5918-GABA',
  'Sample_5643-GABA',
  'Sample_5387-GLU',
  'Sample_5976-GLU',
  'Sample_4725-GLU',
  'Sample_4428-GABA',
  'Sample_1539-GLU',
  'Sample_4332-GABA',
  'Sample_4379-GABA',
  'Sample_1848-GABA',
  'Sample_179134-1401-GABA',
  'Sample_Hct16HCQA020-GLU',
  'Sample_5643-GLU',
  'Sample_4321-GLU',
  'Sample_5744-GABA',
  'Sample_5401-GABA',
  'Sample_5293-GABA',
  'S

In [34]:
os.chdir('/cndd3/mcuoco/projects/czi_diversity_rnate/results/merge_bam/')
curr_l = glob.glob('**')
bam_ll = []
for curr in curr_l:
    bam_l = glob.glob(curr+'/*.bam')
    print(len(bam_l))
    bam_ll = np.append(bam_ll, bam_l)

19
19
19
19
18
19
19
19
19
19
19
19
19
19
18
19
18
19
19
19
19


In [8]:
import os
import glob
os.chdir('/cndd3/dburrows/DATA/te/rna/PE.bam')
l = glob.glob('*Samp**')
for n in l:
    os.mkdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + n)

In [6]:
l

['Sample_5570-GABA',
 'Sample_5086-GLU',
 'Sample_1823-GABA',
 'Sample_6285-GABA',
 'Sample_1277-GABA',
 'Sample_5976-GABA',
 'Sample_5161-GABA',
 'Sample_4411-GABA',
 'Sample_4369-GLU',
 'Sample_5326-GABA',
 'Sample_HctYPA008-GABA',
 'Sample_5309-GLU',
 'Sample_4411-GLU',
 'Sample_5871-GLU',
 'Sample_5077-GABA',
 'Sample_198034-1435-GABA',
 'Sample_5086-GABA',
 'Sample_Hct15HBNA032-GABA',
 'Sample_5936-GLU',
 'Sample_4337-GABA',
 'Sample_1241-GLU',
 'Sample_5184-GABA',
 'Sample_5918-GABA',
 'Sample_5643-GABA',
 'Sample_5387-GLU',
 'Sample_5976-GLU',
 'Sample_4725-GLU',
 'Sample_4428-GABA',
 'Sample_1539-GLU',
 'Sample_4332-GABA',
 'Sample_4379-GABA',
 'Sample_1848-GABA',
 'Sample_179134-1401-GABA',
 'Sample_Hct16HCQA020-GLU',
 'Sample_5643-GLU',
 'Sample_4321-GLU',
 'Sample_5744-GABA',
 'Sample_5401-GABA',
 'Sample_5293-GABA',
 'Sample_1133-GLU',
 'Sample_1134-GLU',
 'Sample_4425-GLU',
 'Sample_3739-GLU',
 'Sample_6285-GLU',
 'Sample_6007-GABA',
 'Sample_4425-GABA',
 'Sample_1133-GABA

In [2]:
import pandas as pd
bed_pl = pd.read_csv("/cndd3/dburrows/DATA/te/gtf/bed/rmsk.hg38.filt-5ptrim.plus.sort.bed", sep='\t', header=None)



In [4]:
bed_pl

,0,1,2,3,4,5,6,7
0,chr1,24038,24138,+,MIR,MIR,SINE,211
1,chr1,26741,26841,+,AluSp,Alu,SINE,263
2,chr1,31386,31486,+,AluJo,Alu,SINE,297
3,chr1,32791,32891,+,MIR,MIR,SINE,214
4,chr1,39574,39674,+,AluSx,Alu,SINE,300
...,...,...,...,...,...,...,...,...
548906,chrY,57200662,57200762,+,AluSg7,Alu,SINE,308
548907,chrY,57201438,57201538,+,AluSp,Alu,SINE,305
548908,chrY,57202215,57202315,+,AluSz,Alu,SINE,278
548909,chrY,57202519,57202619,+,AluSc,Alu,SINE,308


In [7]:
bed_pl.columns =['Chromosome', 'Start', 'End', 'Strand', 'gene_id', 'family_id', 'class_id', 'length']
bed_pl

,Chromosome,Start,End,Strand,gene_id,family_id,class_id,length
0,chr1,24038,24138,+,MIR,MIR,SINE,211
1,chr1,26741,26841,+,AluSp,Alu,SINE,263
2,chr1,31386,31486,+,AluJo,Alu,SINE,297
3,chr1,32791,32891,+,MIR,MIR,SINE,214
4,chr1,39574,39674,+,AluSx,Alu,SINE,300
...,...,...,...,...,...,...,...,...
548906,chrY,57200662,57200762,+,AluSg7,Alu,SINE,308
548907,chrY,57201438,57201538,+,AluSp,Alu,SINE,305
548908,chrY,57202215,57202315,+,AluSz,Alu,SINE,278
548909,chrY,57202519,57202619,+,AluSc,Alu,SINE,308


In [8]:
#drop last columns
bed_pl = bed_pl.drop(columns=['length'])
bed_pl

,Chromosome,Start,End,Strand,gene_id,family_id,class_id
0,chr1,24038,24138,+,MIR,MIR,SINE
1,chr1,26741,26841,+,AluSp,Alu,SINE
2,chr1,31386,31486,+,AluJo,Alu,SINE
3,chr1,32791,32891,+,MIR,MIR,SINE
4,chr1,39574,39674,+,AluSx,Alu,SINE
...,...,...,...,...,...,...,...
548906,chrY,57200662,57200762,+,AluSg7,Alu,SINE
548907,chrY,57201438,57201538,+,AluSp,Alu,SINE
548908,chrY,57202215,57202315,+,AluSz,Alu,SINE
548909,chrY,57202519,57202619,+,AluSc,Alu,SINE


In [1]:
#Check that all files are present
import glob
import os

full_l = os.listdir('/cndd2/Public_Datasets/Dracheva_PsychEncode_development/raw_May2022/') 
os.chdir('/cndd3/dburrows/DATA/te/rna/PE.bam/')
check_l = glob.glob('*Samp*')

import numpy as np
not_int = np.setxor1d(check_l, full_l)
assert len(full_l) == len(check_l)
assert len(not_int) == 0

In [ ]:
STAR --genomeDir /cndd2/jchien/iGenome/STAR_gencode_v37 --readFilesIn  R1-merge.fastq.gz R1-merge.fastq.gz --outSAMunmapped None --outFilterType BySJout --outSAMattributes All --outFilterMultimapNmax 100 --outFilterMismatchNmax 999 --outFilterMismatchNoverReadLmax 0.04 --alignIntronMin 20 --alignIntronMax 1000000 --alignMatesGapMax 1000000 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 --sjdbScore 1 --readFilesCommand zcat --runThreadN 16--genomeLoad LoadAndKeep --limitBAMsortRAM 10000000000 --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --winAnchorMultimapNmax 200 --outMultimapperOrder Random --outSAMmultNmax -1

In [76]:
# Check which files went to completion
import glob
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/')
curr_ = glob.glob('*Sa*')
for c in curr_:
    os.chdir(c)
    atem = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + c + '/*ATEM*')
    log = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + c + '/*log*')
    if not atem or not log:
        print(c)
    if log:
        if 'All reads accounted for' not in np.load(log[0]):
            print(c)

    os.chdir('..')